# Covid-19 vs Mortality Rates By County

(Preface: I am not a medical professional and should not be taken for medical advice. This notebook was created as part of a data science project through Udacity) 
